# Zadanie 2

In [ ]:
nr_tries = 5

## Rekurenycjne odwracanie macierzy

In [ ]:
import numpy as np

def multi_dot(dot_m, arrays): 
    '''
    function used to make multiple dots in one go
    '''
    result, op = dot_m(arrays[0], arrays[1])
    for i in range(2, len(arrays)):
        result, op_new = dot_m(result, arrays[i])
        op+=op_new
    return result, op


def recursive_inversion(matrix, dot_method):   
    '''
    function returns inverted matrix and operations it took to invert
    '''
    cnt = 0
    if matrix.shape[0] != matrix.shape[1]: raise Exception('Matrix is not a square')
    
    if matrix.shape == (1, 1):
        if matrix[0, 0] == 0: raise Exception('Matrix is invertible')
        cnt += 1
        return np.array([[1/matrix[0, 0]]]), cnt
    
    elif matrix.shape == (2, 2):
        det = matrix[0, 0]*matrix[1, 1] - matrix[0, 1]*matrix[1, 0]
        if det == 0: raise Exception('Matrix is invertible')
        cnt += 6
        return np.array([[matrix[1, 1]/det, -matrix[0, 1]/det], [-matrix[1, 0]/det, matrix[0, 0]/det]]), cnt
    
    
    n = matrix.shape[0]
    m = n//2
    A = matrix[:m, :m]
    B = matrix[:m, m:]
    C = matrix[m:, :m]
    D = matrix[m:, m:]
    
    A_inv, cnt1 = recursive_inversion(A, dot_method)
    Mul_Prod_1, mp_cnt_1 = multi_dot(dot_method, [C, A_inv, B])
    Common, cnt2 = recursive_inversion(D - Mul_Prod_1, dot_method)
    # zwrócić uwagę na D-Mul_Prod

    Mul_Prod_2, mp_cnt_2 =multi_dot(dot_method, [A_inv, B, Common, C, A_inv])
    P = A_inv+Mul_Prod_2
    # zwrócić uwagę na A_inv+Mul_Prod_2

    Mul_Prod_3, mp_cnt_3 =multi_dot(dot_method, [A_inv, B, Common])
    Q = -Mul_Prod_3
    Mul_Prod_4, mp_cnt_4 = multi_dot(dot_method, [Common, C, A_inv])
    R = -Mul_Prod_4

    S = Common  

    Top = np.row_stack([P, R])
    Bottom = np.row_stack([Q,S])
    cnt += cnt1 + cnt2 + mp_cnt_1 + mp_cnt_2 + mp_cnt_3 + mp_cnt_4

    return np.column_stack([Top, Bottom]), cnt
    



In [ ]:
def assert_same(A, B, sigma = 1e-2):
    for i,row in enumerate(A):
        for j, el in enumerate(row):
            assert abs(el-B[i][j]) < sigma

In [ ]:
import time
from matplotlib import pyplot as plt
from multi_algorithms import cauchy_binet_recursive, strassen_matrix_multiply


def test_algorithm(max_k):
    x = []
    y_t = [[] for _ in range(2)]
    y_op = [[] for _ in range(2)]
    for k in range(2,max_k+1):
        n = 2**k
        A = np.random.rand(n,n)

        start = time.time()
        A_inv, ops = recursive_inversion(A, cauchy_binet_recursive)
        end = time.time()

        assert_same(A_inv, np.linalg.inv(A))
        y_t[0].append(end-start)
        y_op[0].append(ops)

        start = time.time()
        A_inv, ops = recursive_inversion(A, strassen_matrix_multiply)
        end = time.time()

        assert_same(A_inv, np.linalg.inv(A))
        y_t[1].append(end-start)
        y_op[1].append(ops)

        x.append(k)


    return x, y_t, y_op


Tests

In [ ]:
x, y_t, y_op = test_algorithm(nr_tries)


In [ ]:
# plot two bars on one plot
plt.figure(figsize=(14,7))
plt.subplot(1,2,1)
plt.suptitle('Time of recursive inversion depending on chosen multiplication algorithm:')
plt.bar(x, y_t[0])
plt.title('Cauchy-Binet')
plt.xlabel('k')
plt.ylabel('Time[s]')
plt.ylim(top=65)
plt.subplot(1,2,2)
plt.bar(x, y_t[1], color="green")
plt.title('Strassen')
plt.xlabel('k')
plt.ylabel('Time[s]')
plt.ylim(top=65)

print([str(round(el,2)) + 's' for el in y_t[0]])
print([str(round(el,2)) + 's' for el in y_t[1]])


In [ ]:
# plot two bars on one plot
plt.figure(figsize=(14,7))
plt.subplot(1,2,1)
plt.suptitle('Operations of recursive inversion depending on chosen multiplication algorithm:')
plt.bar(x, y_op[0])
plt.title('Cauchy-Binet')
plt.xlabel('k')
plt.ylabel('number of operations')
plt.subplot(1,2,2)
plt.ylim(top=8e7)
plt.bar(x, y_op[1], color = "green")
plt.title('Strassen')
plt.xlabel('k')
plt.ylabel('number of operations')
plt.ylim(top=8e7)

print(y_op[0])
print(y_op[1])

## LU faktoryzacja

In [ ]:
def lu_decomposition(A, dot_method):
    n = len(A)
    
    if n == 1:
        L = np.array([[1]])
        U = A
        return L, U, 0

    A11 = A[:n//2, :n//2]
    A12 = A[:n//2, n//2:]
    A21 = A[n//2:, :n//2]
    A22 = A[n//2:, n//2:]

    L11, U11, count1 = lu_decomposition(A11, dot_method)
    U11_inv, count2 = recursive_inversion(U11, dot_method)
    L21, count3 = dot_method(A21, U11_inv)
    L11_inv, count4 = recursive_inversion(L11, dot_method)
    U12, count5 = dot_method(L11_inv, A12)
    A21U11_inv, count6 = dot_method(A21, U11_inv)
    L11_invA12, count7 = dot_method(L11_inv, A12)
    tmp, count8 = dot_method(A21U11_inv, L11_invA12)
    S, count9 = A22 - tmp, tmp.shape[0]*tmp.shape[0]
    # Zwrócić uwagę A22 - tmp
    Ls, Us, count10 = lu_decomposition(S, dot_method)
    U22 = Us
    L22 = Ls

    L1 = np.hstack((L11, np.zeros((n//2, n//2))))
    L2 = np.hstack((L21, L22))
    L = np.vstack((L1, L2))

    U1 = np.hstack((U11, U12))
    U2 = np.hstack((np.zeros((n//2, n//2)), U22))
    U = np.vstack((U1, U2))
    
    count = count1 + count2 + count3 + count4 + count5 + count6 + count7 + count8 + count9 + count10

    return L, U, count

In [ ]:
def test_lu(max_k):
    x = []
    y_t = [[] for _ in range(2)]
    y_op = [[] for _ in range(2)]
    for k in range(2,max_k+1):
        n = 2**k
        A = np.random.rand(n,n)

        start = time.time()
        L, U, ops = lu_decomposition(A, cauchy_binet_recursive)
        end = time.time()
         
        assert_same(A, L @ U)
        y_t[0].append(end-start)
        y_op[0].append(ops)

        start = time.time()
        L, U, ops = lu_decomposition(A, strassen_matrix_multiply)
        end = time.time()

        assert_same(A, L @ U)
        y_t[1].append(end-start)
        y_op[1].append(ops)

        x.append(k)


    return x, y_t, y_op

Testy

In [ ]:
x, y_t, y_op = test_lu(nr_tries)

In [ ]:
# plot two bars on one plot
plt.figure(figsize=(14,7))
plt.subplot(1,2,1)
plt.suptitle('Time of LU decomposition depending on chosen multiplication algorithm:')
plt.bar(x, y_t[0])
plt.title('Cauchy-Binet')
plt.xlabel('k')
plt.ylabel('Time[s]')
plt.ylim(top=60)
plt.subplot(1,2,2)
plt.bar(x, y_t[1], color = "green")
plt.title('Strassen')
plt.xlabel('k')
plt.ylabel('Time[s]')
plt.ylim(top=60)


print([str(round(el,2)) + 's' for el in y_t[0]])
print([str(round(el,2)) + 's' for el in y_t[1]])

In [ ]:
# plot two bars on one plot
plt.figure(figsize=(14,7))
plt.subplot(1,2,1)
plt.suptitle('Operations of LU decomposition depending on chosen multiplication algorithm:')
plt.bar(x, y_op[0])
plt.title('Cauchy-Binet')
plt.xlabel('k')
plt.ylabel('number of operations')
plt.subplot(1,2,2)
plt.ylim(top=7e7)
plt.bar(x, y_op[1], color = "green")
plt.title('Strassen')
plt.xlabel('k')
plt.ylabel('number of operations')
plt.ylim(top=7e7)

print(y_op[0])
print(y_op[1])

## Wyznacznik

In [ ]:
def det(A, dot_method):
    
    L, U, count = lu_decomposition(A, dot_method)

    det_L = np.prod(np.diagonal(L))  
    det_U = np.prod(np.diagonal(U))  

    return det_U*det_L, count + 2*U.shape[0] + 1

In [ ]:
def test_det(max_k, sigma=10e-4):
    x = []
    y_t = [[] for _ in range(2)]
    y_op = [[] for _ in range(2)]
    for k in range(2,max_k+1):
        print(f"Tests for k = {k}:")
        n = 2**k
        A = np.random.rand(n,n)

        start = time.time()
        detA, ops = det(A, cauchy_binet_recursive)
        end = time.time()

        res1 = detA - np.linalg.det(A)
        print("{:.2e}".format(res1), "ch", end= " \t")

        y_t[0].append(end-start)
        y_op[0].append(ops)

        start = time.time()
        detA, ops = det(A, strassen_matrix_multiply)
        end = time.time()

        res2 = detA - np.linalg.det(A)
        print("{:.2e}".format(res2), "st")

        y_t[1].append(end-start)
        y_op[1].append(ops)

        x.append(k)


    return x, y_t, y_op

Testy

In [ ]:
x, y_t, y_op = test_det(nr_tries)

In [ ]:
# plot two bars on one plot
plt.figure(figsize=(14,7))
plt.subplot(1,2,1)
plt.suptitle('Time of determinant depending on chosen multiplication algorithm:')
plt.bar(x, y_t[0])
plt.title('Cauchy-Binet')
plt.xlabel('k')
plt.ylabel('Time[s]')
plt.ylim(top=60)
plt.subplot(1,2,2)
plt.bar(x, y_t[1], color = "green")
plt.title('Strassen')
plt.xlabel('k')
plt.ylabel('Time[s]')
plt.ylim(top=60)

print([str(round(el,2)) + 's' for el in y_t[0]])
print([str(round(el,2)) + 's' for el in y_t[1]])

In [ ]:
# plot two bars on one plot
plt.figure(figsize=(14,7))
plt.subplot(1,2,1)
plt.suptitle('Operations of determinant depending on chosen multiplication algorithm:')
plt.bar(x, y_op[0])
plt.title('Cauchy-Binet')
plt.xlabel('k')
plt.ylabel('number of operations')
plt.ylim(top=7e7)
plt.subplot(1,2,2)
plt.bar(x, y_op[1], color = "green")
plt.title('Strassen')
plt.xlabel('k')
plt.ylabel('number of operations')
plt.ylim(top=7e7)


print(y_op[0])
print(y_op[1])

In [ ]:
# test inv and det on simple ex
A = np.array([[6, 4],[ 2, 9]])
print(f"inputed Array\n", A)
print("Inv array:\n", recursive_inversion(A, cauchy_binet_recursive)[0])
print("Det array: ",det(A, strassen_matrix_multiply)[0])
